In [3]:
# Import required libraries
from datetime import datetime as dt
import json
import os
from pathlib import Path
from typing import List
from dotenv import load_dotenv
import pandas as pd
from pandas import DatetimeIndex
from requests import request

# sample url for historical request
# https://api.aerisapi.com/conditions/80210?from=2016-05-04%2000:00:00&to=2016-05-04%2023:59:59&limit=24&client_id=[CLIENT_ID]&client_secret=[CLIENT_SECRET]


In [4]:
# Load .env environment variables
load_dotenv()

True

In [8]:
# import API keys
client_id = os.getenv("AERIS_CLIENT_ID")
client_secret = os.getenv("AERIS_CLIENT_SECRET")

# print type of API key
print(type(client_id))

# define which location you'd like to receive data from (you can insert multiple locations with (,) seperator)
location_list = [
    "new+york,ny"
]

# Create empty request list
request_fields = []

# edit section below to specify weather attributes for your use case or remove it to use all returned values
request_fields = [
    'periods.dateTimeISO',
    'place.name',
    'place.country',
    'periods.tempF',
    'periods.feelslikeF',
    'periods.humidity',
]

<class 'str'>


In [37]:
#define start date of date range and how many periods back you would like to receive data from. 
dt_list = pd.date_range(start='2022-06-01', periods=2)

# print the data range list. 
print(dt_list)

DatetimeIndex(['2022-06-01', '2022-06-02'], dtype='datetime64[ns]', freq='D')


In [38]:
def aeris_api_dataframe(location: str, custom_fields: List[str] = None, from_date: dt = None) -> pd.DataFrame:
    formatted_fields = []

    if custom_fields is not None:
        formatted_fields = ','.join(custom_fields)

    print(f"retrieving data for {location} on {from_date.strftime('%Y-%m-%d')}...")
    res = request(
        method="GET",
        url=f"https://api.aerisapi.com/conditions/{location}",
        params={
            "client_id": client_id,
            "client_secret": client_secret,
            "fields": formatted_fields,
            #"from": from_date, (orignal code)
            "from": from_date.strftime('%Y-%m-%d 12:59:59'),
            #"to": from_date.strftime('%Y-%m-%d 23:59:59'), (original code)
            "to": from_date.strftime('%Y-%m-%d 13:59:59'),
            #Amount of measurements per day (niels)
            "limit": 1
        }
    )
    
    if res.status_code != 200:
        raise Exception(f"status code was not 200: {res.status_code}")
          
    api_response_body = json.loads(res.text)

    try:
        df_pre_period = pd.json_normalize(api_response_body['response'][0]).drop("periods", axis=1)
        df_periods = pd.json_normalize(api_response_body['response'][0], "periods", record_prefix="periods.")
        return df_pre_period.join(df_periods, how="cross")
    except IndexError:
        print(f"API Response did not contain periods. Verify request parameters are correct.\n\nRequest:\n{res.url}\n\nResponse:\n{api_response_body}")

In [39]:
# Set location where to place the data
output_dir = Path('.//Resources/')
# create location if it doesn't exist
output_dir.mkdir(exist_ok=True)

In [57]:
# get data from AERIS as per datelist
def loop_dates(dt_list: DatetimeIndex):
    sorted_dates = dt_list.sort_values()
    
    # get locations from location_list
    for loc in location_list:
        
        # create file name
        filename = f"weather_{loc}_download.csv"
        
        # create empty list
        all_dates = []
        
        # go through all dates in sorted_dates
        for date in sorted_dates:
            
            # print location and date that will be extracted from api data
            # print(loc,date)
            
            # run aeris_api_dataframe with location, request_files and dates
            all_dates.append(aeris_api_dataframe(location=loc, custom_fields=request_fields, from_date=date))
            # combine all data in all_data_df dataframe
            all_days_df = pd.concat(all_dates, ignore_index=True)
            
            # print all_days_df
            #print(all_days_df)
        
        #save all_days_df to a csv file using output_dir and filename
        all_days_df.to_csv(output_dir / filename, encoding="utf-8")
        
        # print where file for certain location is saved as a csv
        print(f"csv for {loc} saved as {output_dir / filename}")
    
    # print that process has finished. 
    print("all csv's complete!")

loop_dates(dt_list)

new+york,ny 2022-06-01 00:00:00
retrieving data for new+york,ny on 2022-06-01...
  place.name place.country        periods.dateTimeISO  periods.tempF  \
0   new york            us  2022-06-01T12:00:00-04:00          71.29   

   periods.feelslikeF  periods.humidity  
0               71.29                62  
new+york,ny 2022-06-02 00:00:00
retrieving data for new+york,ny on 2022-06-02...
  place.name place.country        periods.dateTimeISO  periods.tempF  \
0   new york            us  2022-06-01T12:00:00-04:00          71.29   
1   new york            us  2022-06-02T12:00:00-04:00          73.87   

   periods.feelslikeF  periods.humidity  
0               71.29                62  
1               73.87                75  
csv for new+york,ny saved as Resources\weather_new+york,ny_download.csv
all csv's complete!
